In [1]:
!cypress-kinit

In [ ]:
!cat ./data/ch2oh_dump_1ns.lammpstrj | head -n 2030 > ch2oh_short.lammpstrj

In [ ]:
! head -n 2030 ./data/ch2oh_dump_1ns.lammpstrj  > ch2oh_short.lammpstrj 

In [ ]:
!hdfs dfs -copyFromLocal ./ch2oh_short.lammpstrj 677project/data

In [2]:
%%writefile codes/Mapper01.py
#!/usr/bin/env python

import sys
import string
import math
import re


class Atom(object):

    def __init__(self, atom_id,
                 atom_type, atom_x, atom_y, atom_z):
        self.atom_id = atom_id
        self.atom_type = atom_type
        self.atom_x = atom_x
        self.atom_y = atom_y
        self.atom_z = atom_z

class BoxBounds(object):

    def __init__(self, bounds_xyz1,
                 bounds_xyz2, bounds_xyz3):
        self.bounds_xyz1 = bounds_xyz1
        self.bounds_xyz2 = bounds_xyz2
        self.bounds_xyz3 = bounds_xyz3

class Calculation(object):

    rC = [5]                                                                                                           
    rHS = [3, 4]                                                                                                            
    rOS = [6]
    rHW = [1]
    rOW = [2]

    def __init__(self, a, b, c, cosalpha, cosbeta, cosgamma):
        self.a = a
        self.b = b
        self.c = c
        self.cosalpha = cosalpha
        self.cosbeta = cosbeta
        self.cosgamma = cosgamma
       

    def init(self, items):
        temp = []
        coords_replica = {}  # the real coords that are used, one of the 9 replica
        x = []
        tmp = []
        OW = []
        h1 = h2 = o1 = o2 = [None] * 100000   # at least 8 times larger than one box h2o atoms.
        #for j in range(len(items)):
         #   temp = [float(n) for n in items[j]]
          #  x.append(temp)
        x = items

        #for i in range(len(x)):
         #   x[i][0] = int(x[i][0])
          #  x[i][1] = int(x[i][1])
           # x[i][2] = x[i][2] #+ 1 - x[0][2]
            #x[i][3] = x[i][3] #- x[0][3]
            #x[i][4] = x[i][4] #- x[0][4]

        h1 = self.findNonperiodic (self.rHS,x)   # item# and x, y, z for each h1 or h2 or o1 or o2
        h2 = self.find (self.rHW,x)   # item# and x, y, z for each h1 or h2 or o1 or o2
        o1 = self.findNonperiodic (self.rOS,x)   # item# and x, y, z for each h1 or h2 or o1 or o2
        o2 = self.find (self.rOW,x)   # item# and x, y, z for each h1 or h2 or o1 or o2
        h1 = [k for k in h1 if k != None]  #h1
        h2 = [k for k in h2 if k != None]  #h2
        o1 = [k for k in o1 if k != None]  #o1
        o2 = [k for k in o2 if k != None]  #o2
        
        leno1 = len(o1)
        leno2 = len(o2)

        #print("abc",self.a, self.b, self.c, self.cosalpha, self.cosbeta, self.cosgamma)

        for i in range(len(o1)):
            for j in range(len(o2)):
                do1o2 = self.dist(o1[i], o2[j])
                #print(do1o2)
                if do1o2 <= float(12.25):
                    for l in range(len(h2)):
                        do2h2 = self.dist(o2[j],h2[l])
                        if do2h2<=1.2:
                            do1h2 = self.dist(o1[i] , h2[l])
                            A1 = self.ang( do1o2 , do1h2, do2h2)
                            A2 = self.ang( do2h2 , do1h2, do1o2)
                            # if A2>=120:
                            if A1<=30 and A2>=120 and do1h2<=6.25:
                                for n in range(len(h2)):
                                    if self.dist(o2[j], h2[n])<=1.2:
                                        tmp.append((o2[j][4],h2[n][4]))
                                        if (o2[j][4] not in coords_replica.keys()):
                                            coords_replica[o2[j][4]] = (o2[j][1],o2[j][2],o2[j][3])  # save found O, H coords, x y z
                                        if (h2[n][4] not in coords_replica.keys()):
                                            coords_replica[h2[n][4]] = (h2[n][1],h2[n][2],h2[n][3])  # save found O, H coords, x y z
                    for l in range(len(h1)):
                        do1h1 = self.dist(o1[i],h1[l])
                        if do1h1<=1.2:
                            do2h1 = self.dist(o2[j] , h1[l])
                            # print do1h1,do1o2,do2h1
                            A1 = self.ang( do1o2 , do2h1, do1h1 )
                            A2 = self.ang( do1h1 ,do2h1, do1o2 )
                            # if  A2>=120:
                            if  A1<=30 and A2>=120 and do2h1<=6.25:
                                for m in range(len(h2)):
                                    if self.dist(o2[j],h2[m])<=1.2:  # output HBed Ow and corresponed Hw. At least two pairs, Ow-Hw1 AND Ow-Hw2
                                        tmp.append((o2[j][4],h2[m][4])) # o2[j][4] is j line with item# ([4], already reordered
                                        if (o2[j][4] not in coords_replica.keys()):
                                            coords_replica[o2[j][4]] = (o2[j][1],o2[j][2],o2[j][3])  # save found O, H coords, x y z
                                        if (h2[m][4] not in coords_replica.keys()):
                                            coords_replica[h2[m][4]] = (h2[m][1],h2[m][2],h2[m][3])  # save found O, H coords, x y z
        tmp = list(set(tmp))   # convert 2 lines(2 pairs in 1 h2o mlc: Ow-Hw1 AND Ow-Hw2) to 1 lines: Ow Hw1 Hw2, thus deletes repeating Ow
        for p in range(len(tmp)):
            for q in range(p+1,len(tmp)):  # set the repeated value to -1
                if(tmp[p][0] == tmp[q][0] and tmp[p][1] == tmp[q][1]):
                    tmp[q][0]= -1
            for t in range(p+1, len(tmp)):
                if(tmp[t][0]>0 and tmp[p][0] == tmp[t][0]):
                    OW.append((tmp[p][0],tmp[p][1],tmp[t][1]))
                    #print tmp[p][0],tmp[p][1],tmp[t][1]
                #    print (coords_replica)
        return OW, coords_replica

    def dist(self, X, Y):
        x1 = X[1]
        x2 = Y[1]
        y1 = X[2]
        y2 = Y[2]
        z1 = X[3]
        z2 = Y[3]
        return 2*(x1-x2)*(y1-y2)*self.a*self.b*self.cosgamma + \
               2*(y1-y2)*(z1-z2)*self.b*self.c*self.cosalpha + \
               2*(x1-x2)*(z1-z2)*self.a*self.c*self.cosbeta + \
               (x1-x2)*(x1-x2)*self.a*self.a + \
               (y1-y2)*(y1-y2)*self.b*self.b + (z1-z2)*(z1-z2)*self.c*self.c

    def ang(self,d1,d2,d3):# Note: square of the distances
        return math.degrees(math.acos((d1 + d2 - d3)/(2.0 * math.sqrt(d1) * math.sqrt(d2))))

    def findNonperiodic(self, y, atoms):
        x=grab= [None] * 100000
        i = 0
        for atom in atoms:
            grab = ((atom.atom_type,atom.atom_x,atom.atom_y,atom.atom_z,atom.atom_id))
            for j in range(len(y)):
                if grab[0] == y[j]:
                    x[i]=grab
            i += 1
        #print("findNon x:", x)
        return (x)


        #below is periodic boundary #
    def find(self,y,atoms):
        x=grab= grabxp = grabxn = grabyp = grabyn = grabxpyp = grabxnyn = grabxnyp = grabxpyn = [None] * 100000
        i = 0
        for atom in atoms:
            grab = ((atom.atom_type,atom.atom_x,atom.atom_y,atom.atom_z,atom.atom_id))
            grabxp = ((atom.atom_type,atom.atom_x + 1,atom.atom_y,atom.atom_z,atom.atom_id))
            grabxn = ((atom.atom_type,atom.atom_x - 1,atom.atom_y,atom.atom_z,atom.atom_id))
            grabyp = ((atom.atom_type,atom.atom_x,atom.atom_y + 1,atom.atom_z,atom.atom_id))
            grabyn = ((atom.atom_type,atom.atom_x,atom.atom_y - 1,atom.atom_z,atom.atom_id))
            grabxpyp = ((atom.atom_type,atom.atom_x + 1,atom.atom_y + 1,atom.atom_z,atom.atom_id))
            grabxpyn = ((atom.atom_type,atom.atom_x + 1,atom.atom_y - 1,atom.atom_z,atom.atom_id))
            grabxnyp = ((atom.atom_type,atom.atom_x - 1,atom.atom_y + 1,atom.atom_z,atom.atom_id))
            grabxnyn = ((atom.atom_type,atom.atom_x - 1,atom.atom_y - 1,atom.atom_z,atom.atom_id))
            for j in range(len(y)):
                if grab[0] == y[j]:
                    x[9*i+0]=grab
                    x[9*i+1]=grabxp
                    x[9*i+2]=grabxn
                    x[9*i+3]=grabyp
                    x[9*i+4]=grabyn
                    x[9*i+5]=grabxpyp
                    x[9*i+6]=grabxpyn
                    x[9*i+7]=grabxnyp
                    x[9*i+8]=grabxnyn
            i += 1
        #print("find x:", x)
        return (x)

    def coor(self, xlo, xhi, xy, ylo, yhi, xz, zlo, zhi, yz):  # calculate a,b,c,cosalpha,cosbeta,cosgamma
        data = [None] * 6
        xlo_bound = float(xlo)
        xhi_bound = float(xhi)
        xy =  float(xy)
        ylo_bound =  float(ylo)
        yhi_bound =  float(yhi)
        xz =  float(xz)
        zlo =  float(zlo)
        zhi =  float(zhi)
        yz =  float(yz)
        xlo = xlo_bound - min(0.0,xy,xz,xy+xz)
        xhi = xhi_bound - max(0.0,xy,xz,xy+xz)
        ylo = ylo_bound - min(0.0,yz)
        yhi = yhi_bound - max(0.0,yz)
        lx = xhi-xlo
        ly = yhi-ylo
        lz = zhi-zlo
        self.a = lx   #a
        self.b = math.sqrt(ly*ly+xy*xy)   #b
        self.c = math.sqrt(lz*lz+xz*xz+yz*yz)  #c
        self.cosalpha = (xy*xz+ly*yz)/self.b/self.c   #cosalpha
        self.cosbeta = xz/self.c    #cosbeta
        self.cosgamma = xy/self.b   #cosgamma

#Main
id = -1
linecount = 0
xyz1 = []
xyz2 = []
xyz3 = []
timestep = -1
num_of_atoms = -1
atoms_list = []

for line in sys.stdin:
    #id += 1
    #print(id, frame)
    
    line = line.strip()
    if linecount == 1:
        timestep = line
    if linecount == 3:
        num_of_atoms = line
    if linecount == 5:
        xyz1 = line.split()
        if len(xyz1) < 4:
            xyz1.append('0.0')
    if linecount == 6:
        xyz2 = line.split()
        if len(xyz2) < 4:
            xyz2.append('0.0')
    if linecount == 7:
        xyz3 = line.split()
        if len(xyz3) < 4:
            xyz3.append('0.0')
    if linecount > 8:
        atom = line.split()
        atoms_list.append(Atom(int(atom[1]), int(atom[2]), float(atom[3]), float(atom[4]), float(atom[5])))
    
    linecount += 1
    
    
    #if id > 202:
        #break
        
    
c = Calculation(0.0,0.0,0.0,0.0,0.0,0.0)#init a,b,c,cosalpha,cosbeta,cosgamma
xlo = xyz1[1]
xhi = xyz1[2]
xy = xyz1[3]
ylo = xyz2[1]
yhi = xyz2[2]
xz = xyz2[3]
zlo = xyz3[1]
zhi = xyz3[2]
yz = xyz3[3]
#do calculation
c.coor(xlo, xhi, xy, ylo, yhi, xz, zlo, zhi, yz)
hb_list, real_coords = c.init(atoms_list)
hb_length = len(hb_list)
    
print ("%s$%s$%s$%s" % (timestep,hb_length, hb_list, real_coords))

Overwriting codes/Mapper01.py


In [ ]:
!cat data/ch2oh_dump_1ns.lammpstrj | python codes/Mapper01.py | sort | python codes/Reducer01.py

In [ ]:
!chmod 777 .

In [3]:
%%writefile codes/Reducer01.py
#!/usr/bin/env python
import sys
import math

num_list = []

for line in sys.stdin:
    line = line.strip()
    elements = line.split("$")
    num_list.append(float(elements[1]))
    print("%s\t%s\t%s\t%s" % (elements[0], elements[1],elements[2], elements[3] ))

avgHB = sum(num_list)/float(len(num_list))

print("%s" % avgHB)
    

Overwriting codes/Reducer01.py


In [1]:
%%time
!hdfs dfs -rm -r 677project/output/hadoopoutput/dump.ch2oh99_prod.lammpstrj
!yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar \
    -input 677project/data/dump.ch2oh99_prod.lammpstrj \
    -inputformat org.apache.hadoop.mapred.lib.NLineInputFormat \
    -jobconf mapreduce.input.lineinputformat.linespermap=1755 \
    -jobconf mapred.reduce.tasks=50 \
    -jobconf mapreduce.job.reduces=50 \
    -jobconf mapreduce.reduce.java.opts=-Xmx4096m \
    -jobconf yarn.nodemanager.resource.memory-mb=4096 \
    -jobconf yarn.scheduler.minimum-allocation-mb=2048 \
    -jobconf yarn.scheduler.maximum-allocation-mb=8192 \
    -jobconf mapreduce.map.memory.mb=8192 \
    -jobconf mapreduce.reduce.memory.mb=4096 \
    -jobconf yarn.app.mapreduce.am.resource.mb=8192 \
    -output 677project/output/hadoopoutput/dump.ch2oh99_prod.lammpstrj \
    -file ./codes/Mapper01.py \
    -mapper Mapper01.py \
    -file ./codes/Reducer01.py \
    -reducer Reducer01.py

rm: `677project/output/hadoopoutput/dump.ch2oh99_prod.lammpstrj': No such file or directory
17/12/02 10:36:03 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
17/12/02 10:36:04 WARN streaming.StreamJob: -jobconf option is deprecated, please use -D instead.
packageJobJar: [./codes/Mapper01.py, ./codes/Reducer01.py] [/usr/hdp/2.6.0.3-8/hadoop-mapreduce/hadoop-streaming-2.7.3.2.6.0.3-8.jar] /hadoop_java_io_tmpdir/streamjob7441441949339454114.jar tmpDir=null
17/12/02 10:36:05 INFO client.AHSProxy: Connecting to Application History server at dscim003.palmetto.clemson.edu/10.125.8.215:10200
17/12/02 10:36:06 INFO client.AHSProxy: Connecting to Application History server at dscim003.palmetto.clemson.edu/10.125.8.215:10200
17/12/02 10:36:06 INFO hdfs.DFSClient: Created HDFS_DELEGATION_TOKEN token 17640 for yli25 on ha-hdfs:dsci
17/12/02 10:36:06 INFO security.TokenCache: Got dt for hdfs://dsci; Kind: HDFS_DELEGATION_TOKEN, Service: ha-hdfs:dsci, I

In [ ]:
!hdfs dfs -ls 677project/output/hadoopoutput/dump.ch2oh66_prod.lammpstrj

!hdfs dfs -cat 677project/output/hadoopoutput/dump.ch2oh66_prod.lammpstrj/part-00000 \
    2>/dev/null | wc -l

!hdfs dfs -cat 677project/output/hadoopoutput/dump.ch2oh66_prod.lammpstrj/part-00000 \
    2>/dev/null | head -n 50

In [ ]:
!hdfs dfs -cat 677project/data/dump.ch2oh99_prod.lammpstrj \
    2>/dev/null | wc -l

In [ ]:
!hdfs dfs -cat 677project/data/dump.ch2oh66_prod.lammpstrj \
    2>/dev/null | tail -n +785 | head

In [ ]:
!hdfs dfs -rm -r 677project/output/output01
!yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar \
    -input 677project/data/dump.co.lammpstrj \
    -output 677project/output/output02 \
    -file ./codes/Mapper01.py \
    -mapper Mapper01.py 

In [ ]:
!hdfs dfs -copyFromLocal ./data/ch2oh_dump_1ns.lammpstrj 677project/data

In [8]:
!hdfs dfs -l 677project/output/sparkoutput

Deleted 677project/output/sparkoutput


In [ ]:
!yarn logs -applicationId application_1510258389716_0046 \
    | grep -v INFO

In [ ]:
    -jobconf mapred.reduce.tasks=10 \
    -jobconf mapreduce.job.reduces=10 \
    -jobconf mapreduce.reduce.java.opts=-Xmx4096m \
    -jobconf yarn.nodemanager.resource.memory-mb=4096 \
    -jobconf yarn.scheduler.minimum-allocation-mb=2048 \
    -jobconf yarn.scheduler.maximum-allocation-mb=8192 \
    -jobconf mapreduce.map.memory.mb=8192 \
    -jobconf mapreduce.reduce.memory.mb=4096 \
    -jobconf yarn.app.mapreduce.am.resource.mb=8192 